In [1]:
import pandas as pd
from collections import Counter
from operator import itemgetter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
import math
from nltk.stem.porter import *
from nltk.corpus import stopwords
import nltk

#nltk.download()
stemmer = PorterStemmer()
vectorizer = CountVectorizer()
tfidf = TfidfTransformer()

In [2]:
def remove_punctuation(text):
    return re.sub(ur"[^\w\d'\s]+",'',text)


def normalize(review):
    stops = set(stopwords.words("english"))
    review = review.lower()
    review = remove_punctuation(review)
    #split_tokens = review.split()
    split_tokens = nltk.word_tokenize(review)
    split_tokens = [w for w in split_tokens if not w in stops]
    tokens = [stemmer.stem(token) for token in split_tokens]
    return tokens

def Bow(reviews):   
    aux_idf = [" ".join(rev) for rev in reviews]
    matrix = vectorizer.fit_transform(aux_idf)
    return matrix

In [3]:
df = pd.read_csv('train.tsv', sep="\t", skiprows =[0], header = None)
df.columns = ["Phraseid", "Sentenceid", "Phrase", "Sentiment"]
df.drop(df.columns[:2], axis = 1, inplace = True)
df.head(10)

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2
2,A series,2
3,A,2
4,series,2
5,of escapades demonstrating the adage that what...,2
6,of,2
7,escapades demonstrating the adage that what is...,2
8,escapades,2
9,demonstrating the adage that what is good for ...,2


In [4]:
reviews = df['Phrase'].tolist()
clasif = df['Sentiment'].tolist()

reviews = [review.decode('utf-8') for review in reviews]
norm_revs = [normalize(review) for review in reviews]
print reviews[150]
print norm_revs[150]

a juicy soap opera
[u'juici', u'soap', u'opera']


In [5]:
count = Bow(norm_revs)
ttfidf = tfidf.fit_transform(count)
##################################################
#calculo de probabilidades a priori
clasif_count = (Counter(clasif).most_common())
clasif_count.sort(key = itemgetter(0))

aux_pp = [elem[1] for elem in clasif_count]
prior_probs = [elem/float(len(reviews)) for elem in aux_pp]
prior_probs

[0.04531590413943355,
 0.1747597078046905,
 0.5099448929898757,
 0.21098936306548763,
 0.058990132000512625]

In [6]:
classifier = MultinomialNB(alpha=1.0, class_prior=prior_probs, fit_prior=True)
nbcls  = classifier.fit(ttfidf, clasif)#aca iba ttfidf

In [7]:
test = pd.read_csv('test.tsv', sep="\t", skiprows =[0], header = None)
test.columns = ["Phraseid", "Sentenceid", "Phrase"]
test.drop(test.columns[1], axis = 1, inplace = True)

test.tail(10)

,Phraseid,Phrase
66282,222343,should have called it Gutterball
66283,222344,have called it Gutterball
66284,222345,called it Gutterball
66285,222346,it Gutterball
66286,222347,Gutterball
66287,222348,"A long-winded , predictable scenario ."
66288,222349,"A long-winded , predictable scenario"
66289,222350,"A long-winded ,"
66290,222351,A long-winded
66291,222352,predictable scenario


In [8]:
tests = test['Phrase'].tolist()
ids = test['Phraseid'].tolist()

norm_test = [normalize(test) for test in tests]
norm_test = [" ".join(test) for test in norm_test]

norm_test[100]

'rhythm'

In [9]:
vec = vectorizer.transform(norm_test)

sents = nbcls.predict(vec)

print sents[0:10]

[3 3 2 3 3 3 3 3 3 2]


In [10]:
data = zip(ids,sents)
ans = pd.DataFrame(data)
ans.columns = ["PhraseId", "Sentiment"]

ans.to_csv('Sentimentsipynb.csv', sep=',', index=False)